In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random

import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols

Remember our fuel efficiency dataset?

In [ ]:
df_cars = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv')
df_cars.dropna(inplace=True)
df_cars.head()

In [ ]:
# get line of best fit
model = ols(formula = 'horsepower ~ weight', data=df_cars)
res = model.fit()
res.summary()

Remember if you are on Colab:

```!pip install matplotlib==3.5```

In [ ]:
# we can get the parameters from the model this way as a pandas Series
res.params

Line of best fit!

In [ ]:
b, m = res.params

# plot the points
plt.scatter(x=df_cars['weight'], y=df_cars['horsepower'])

# plot the line
plt.axline((0, b), slope=m, color='green')

# set limits of the axes (need to do this because we specified the y-int)
plt.xlim(min(df_cars['weight']), max(df_cars['weight']))
plt.ylim(min(df_cars['horsepower']), max(df_cars['horsepower']))
plt.show()

Below we plot the *residual* of a random point with the line of best fit.

In [ ]:
# a random point
idx3 = np.random.randint(low=1, high=len(df_cars)+1, size=1)

# use .item() to get just the value not a Series object
x3 = df_cars.loc[idx3, 'weight'].item()
y3 = df_cars.loc[idx3, 'horsepower'].item()

# plot this point
plt.scatter(x=x3, y=y3)

# plot the line
plt.axline((0, b), slope=m, color='green')

# set limits of the x-axis (need to do this because we specified the y-int)
plt.xlim(min(df_cars['weight']), max(df_cars['weight']))
plt.ylim(min(df_cars['horsepower']), max(df_cars['horsepower']))

# plot the prediction vs. the true value
y_pred = m * x3 + b
plt.plot([x3,x3], [y3, y_pred], color='red')

plt.show()

## Residuals

Assume a linear model

$$
Y = \beta_0 + \beta_1 X
$$

and data $\{(x_1,y_1), \dots, (x_N, y_N)\}$. For a particular $x_i$ let $\hat{y}_i$ be the prediction, i.e. $\beta_0 + \beta_1 x_i$.

- The residuals are given by $\{ \hat{y}_1 - y_i, \hat{y}_2 - y_2, \dots, \hat{y}_N - y_N\}$
- We optimize against the sum of the squared residuals (RSS)

$$
RSS = \sum_{i=1}^N (\hat{y}_i - y_i)^2
$$

- Sometimes you will hear about the **M**ean **S**quared **E**rror, MSE, (or **R**oot **M**ean **S**quared **E**rror, RSME). What happens if we optimize against either of these quantities?

$$
MSE = \frac{\sum_{i=1}^N (\hat{y}_i - y_i)^2}{N} = \frac{RSS}{N}, \,\,\,\,\,\, RMSE = \sqrt{MSE}.
$$

## What are we doing really?

- A **Regression** Model is concerned with how a variable $Y$ varies with respect to another input variable (or variables!) $X$.
- What is the probability distribution of $Y$ for a particular value of $X$?
- So what does *linear* regression really mean?

We assume that our variables are related by the following expression

$$
Y = \beta_0 + \beta_1 X + \epsilon
$$

where we the value of $\epsilon$ is normally distributed with mean 0 for each particular value of $X$.

- What does this mean? Does this make sense for a real-life scenario?

## Assumption 1: Linear Relationship

In [ ]:
# fake data that follows this relationship
m = 1
b = 0

num_pts = 100

# create num_pts points equally spaced from 0 to 100
X = np.linspace(0, 100, num_pts)

# Gaussian noise
mu, sigma = 0, 5
epsilon = np.random.normal(mu, sigma, num_pts)

# adding random noise to each prediction
Y = m * X + b + epsilon

plt.scatter(x=X, y=Y)
plt.show()

- How does changing the standard deviation change the plot?

The **Pearson Correlation Coefficient** is a statistical measure of the <i>strength of the linear relationship</i> between $Y$ and $X$:

$$
\rho = \frac{Cov(X,Y)}{\sigma_X \sigma_Y} = Cor(X,Y)=\frac{\sum((x_i-\overline{x})(y_i-\overline{y})}{\sqrt{(x_i-\overline{x})^2}\sqrt{(y_i-\overline{y})^2}}
$$

"Normalized Covariance of X and Y"

In [ ]:
# outputs the pearson correlation ceofficient between each pair of variables
df_cars.corr()

- What do you notice about this matrix?

In [ ]:
# another way of computing it
np.corrcoef(X, Y)

In [ ]:
# we can also look at the fitted values vs. the residual values
Y_pred = m * X + b
residuals = Y_pred - Y

plt.scatter(Y_pred, residuals)
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()

In [ ]:
# what happens for non-linear data?
X = np.linspace(0, 15, 100)

# Gaussian noise
mu, sigma = 0, 10
epsilon = np.random.normal(mu, sigma, num_pts)

# a parabola
Y2 = X**2 + epsilon

plt.scatter(X, Y2)
plt.show()

In [ ]:
# get the line of best fit
# another version of OLS for numpy data
model = sm.OLS(Y2, sm.add_constant(X), hasconst=True)
res = model.fit()
b, m = res.params

# plot the points
plt.scatter(x=X, y=Y2)

# plot the line
plt.axline((0, b), slope=m, color='green')
plt.show()

In [ ]:
# there is a relationship between the residuals and the value of X!
# oh no!

Y_pred = m * X + b
residuals = Y_pred - Y2

plt.scatter(Y_pred, residuals)
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()

## Assumption 2: Homoskedascity
- The error $\epsilon$ is normally distributed with mean 0. 

In [ ]:
# back to our fake data
m = 1
b = 0

num_pts = 100

# create num_pts points equally spaced from 0 to 100
X = np.linspace(0, 100, num_pts)

# Gaussian noise
mu, sigma = 0, 5
epsilon = np.random.normal(mu, sigma, num_pts)

# adding random noise to each prediction
Y = m * X + b + epsilon

plt.scatter(x=X, y=Y)
plt.show()

In [ ]:
# we can also look at the fitted values vs. the residual values
Y_pred = m * X + b
residuals = Y_pred - Y

plt.hist(residuals)
plt.show()

The following is a Q-Q plot or (Probability Plot since we are comparing against a theoretical distrbution) which plots the ordered residuals against an ordered sample of normally distributed points. 

In [ ]:
fig,ax = plt.subplots(figsize=(10,8))
sm.qqplot(residuals,line='s',ax=ax) 

# Normals go on the x-axis
plt.xlabel("Normal Quantiles", fontsize=16)

# Residuals on the y-axis
plt.ylabel("Residual Quantiles", fontsize=16)


plt.show() 

In [ ]:
# what is epsilon is distributed differently??

# Gaussian noise
epsilon = np.random.exponential(scale=10, size=num_pts)

# adding random noise to each prediction
Y = m * X + b + epsilon

plt.scatter(x=X, y=Y)
plt.axline((0, b), slope=m, color='green')
plt.show()

In [ ]:
Y_pred = m * X + b
residuals = Y_pred - Y

plt.hist(residuals)
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(10,8))
sm.qqplot(residuals,line='s',ax=ax) 

# Normals go on the x-axis
plt.xlabel("Normal Quantiles", fontsize=16)

# Residuals on the y-axis
plt.ylabel("Residual Quantiles", fontsize=16)


plt.show() 

## Let's take a break and look at some real data.
- Use what you've learn to identify two variables in the ```diamonds``` dataset which seems to satisfy these two assumptions and two variables which do not seem to satisfy them. Use visual evidence and statistical evidence to support your claim!
- Food for thought: Will I get the same line of best fit if I use squared residuals vs. absolute value of residuals?

In [ ]:
df_dia = sns.load_dataset('diamonds')
df_dia.head()

## Why Squared Residuals anyways?
* Given the assumptions above, the Gauss-Markov theorem says that the OLS estimator is BLUE (the **B**est **L**inear **U**nbiased **E**stimator).
* The OLS estimator hits the mean for the distribution of $Y$ for each fixed value of $X$.
* Assuming this error distribution, what are the values of $\beta_0$ and $\beta_1$ that maximize the probability that we would see these sample values? This is known as the **M**aximum **L**ikelihood **E**stimate or MLE.

These assumptions are also necessary to do a Hypothesis Test. The default assumption is that there is no linear correlation between two variables.
$$
H_0: \beta_1 = 0 \\
H_1: \beta_1 \neq 0
$$

In [ ]:
# looks like 0 is not in the 95% confidence interval, hooray!
model = ols(formula = 'horsepower ~ weight', data=df_cars)
res = model.fit()
res.summary()

## Multiple Linear Regression
- More variables, more fun

In [ ]:
# let's try to predict horsepower again
# why might we want to use multiple variables?
sns.pairplot(df_cars.drop(columns=['cylinders', 'model_year']))

### Multiple Linear Regression

For input variables $X_1, \dots, X_n$ we have the model

$$
Y = \beta_0 + \beta_1 X_1 + \dots + \beta_n X_n,
$$

or more formally

$$
Y = \beta_0 + \beta_1 X_1 + \dots + \beta_n X_n + \epsilon
$$

where epsilon is normally distributed with mean 0 for any fixed tuple of values. The OLS Estimate is still obtained by minimizing the RSS function.

In [ ]:
# get the OLS estimator
model = ols(formula = 'horsepower ~ weight + displacement', data=df_cars)
res = model.fit()
res.summary()

- What happened to the $R^2$ when we included more variables?

## Assumption 3: No Multicolinearity

In [ ]:
df_cars.drop(columns=['cylinders', 'model_year']).corr()

In [ ]:
# a nicer display
sns.heatmap(df_cars.drop(columns=['cylinders', 'model_year']).corr(), annot=True)

Why might this induce issues?
- Violates model assumptions (can't use statistical rigor).
- Interpreting the results.

### The F-Test
Score for the Hypothesis test: 
$$
H_0 : \beta_1=\beta_2=\cdots =\beta_p=0,\\
H_1 : \text{at least one $\beta_i\neq 0$}.
$$
All our assumptions must be satisfied for this to be valid!

In [ ]:
# how can we interpret this?
model = ols(formula = 'horsepower ~ weight + displacement', data=df_cars)
res = model.fit()
res.summary()

In [ ]:
df_dia.head()

In [ ]:
# let's look at using carat and depth to predict diamond price
# I'm going to take a sample to make this harder!
df = df_dia.sample(n=100).drop(columns=['cut','color','clarity','table', 'x', 'y', 'z'])
df.head()

In [ ]:
sns.heatmap(df.corr(), annot=True)

In [ ]:
model = ols(formula = 'price ~ depth + carat', data=df)
res = model.fit()
res.summary()

- Does this mean that we can conclude that ```depth``` is a good predicter here?

## Categorical Variables

In [ ]:
# back to the penguins!
df_penguins = sns.load_dataset('penguins')
df_penguins.head()

### How can we create a linear model which takes the ```sex``` variable into account?
- One-Hot Encode

In [ ]:
# lambda function time!
# amazing Python feature
df_penguins['is_male'] = df_penguins['sex'].apply(lambda x : int(x == 'Male'))

In [ ]:
df_penguins.head()

Let's create a model here:

$$
Y_{mass} = \beta_0 + \beta_1 X_{flipper} + \beta_2 X_{is\_male}
$$

In [ ]:
model = ols(formula = 'body_mass_g ~ flipper_length_mm + is_male', data=df_penguins)
res = model.fit()
res.summary()

## Lab 2

1. Create a scatter plot with flipper length on the x-axis and body mass on the y-axis. Color the points according to the penguin's sex.
2. Find the line of best fit for the penguins dataset to predict body mass from flipper length. Plot this on the plot from Q1.
3. Find two lines of best fit: one for the male and one for the female penguins. Plot this on the plot from Q1.
4. Take the line of best fit from above which used both flipper length and ```is_male```. Plot this line with $X_{is\_male}=0$ and with $X_{is\_male}=1$. What do you notice?

## Next Time
- Quiz 2 to start class
- We'll talk about interpreting, evaluating, and troubleshooting your model.
- Bootstrapping I hope!